In [3]:
!pip install -q -U datasets > /dev/null
# !pip install transformers
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 210.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 253.5 MB/s eta 0:00:00


In [4]:
from transformers import AutoModelWithLMHead, AutoTokenizer

import random, warnings
warnings.filterwarnings("ignore")


from torch.utils.data import Dataset

In [14]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/modelt5spider")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL", use_fast=False)
# model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


# Inferencing on SPIDER

Using pre-trained T5 model

In [6]:
def get_sql(query):
    
    input_text = "translate English to SQL: %s </s>" % query
    
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0])

In [7]:
def get_sql(query, schema):
    # Concatenate the schema information with the input query text, separated by a special token, such as `<schema>`
    input_text = f"translate English to SQL: {query} <schema> {schema} </s>"

    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                             attention_mask=features['attention_mask'])

    decoded_output = tokenizer.decode(output[0])

    # Remove the <pad> token from the output
    return decoded_output.replace('<pad>', '').strip()

In [8]:
!pip install datasets
from transformers import AutoModelWithLMHead, AutoTokenizer
from datasets import load_dataset
import random, warnings
import nltk
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:



from torch.utils.data import Dataset
import json
from datasets import load_dataset

spider_dataset = load_dataset("spider")
class SpiderDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        db_id = self.data[idx]['db_id']
        schema = self.get_schema(db_id)

        input_text = f"translate English to SQL: {self.data[idx]['question']} <schema> {schema} </s>"
        target_text = self.data[idx]['query']
        encoding = self.tokenizer(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target = self.tokenizer(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten(),
            'data_item': self.data[idx] 
        }

    def get_schema(self, db_id):
        schema_data = next(item for item in tables_data if item['db_id'] == db_id)
        schema = " ".join([f"Table: {table_name} Columns: {', '.join([col_name for _, col_name in schema_data['column_names'] if schema_data['table_names'][table_idx] == table_name])}" for table_idx, table_name in enumerate(schema_data['table_names'])])
        return schema

# Load tables.json
with open('/content/drive/MyDrive/spider/tables.json', 'r') as f:
    tables_data = json.load(f)

train_dataset = SpiderDataset(spider_dataset['train'], tokenizer, max_length=128)
val_dataset = SpiderDataset(spider_dataset['validation'], tokenizer, max_length=128)

  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
gold_file = open('gold.txt', 'w')
pred_file = open('pred.txt', 'w')

num_queries = 100
count = 0
for idx in random.sample(range(len(val_dataset)), num_queries):
    item = val_dataset[idx]
    data_item = item['data_item']
    print(f'{count + 1}/{num_queries}')
    print(f"Text: {data_item['question']}")

    # Get schema information
    db_id = data_item['db_id']
    schema = val_dataset.get_schema(db_id)

    pred = get_sql(data_item['question'], schema)
    gold = data_item['query']

    gold_file.write(gold + '\t' + db_id + '\n')
    pred_file.write(pred + '\n')
    
    print(f"Pred SQL: {pred}")
    print(f"True SQL: {gold}\n")

    count += 1
  
gold_file.close()
pred_file.close()


Token indices sequence length is longer than the specified maximum sequence length for this model (517 > 512). Running this sequence through the model will result in indexing errors


1/100
Text: What are the different years in which there were cars produced that weighed less than 4000 and also cars that weighted more than 3000 ?
Pred SQL: SELECT DISTINCT YEAR FROM automobiles AS t1 JOIN country
True SQL: select distinct year from cars_data where weight between 3000 and 4000;

2/100
Text: List the names of poker players ordered by the final tables made in ascending order.
Pred SQL: SELECT T1.name FROM Poker_players AS T1 JOIN players AS
True SQL: SELECT T1.Name FROM people AS T1 JOIN poker_player AS T2 ON T1.People_ID  =  T2.People_ID ORDER BY T2.Final_Table_Made

3/100
Text: Find the id, last name and cell phone of the professionals who live in the state of Indiana or have performed more than two treatments.
Pred SQL: SELECT T1.id, T2.last_name, T2.last
True SQL: SELECT professional_id ,  last_name ,  cell_number FROM Professionals WHERE state  =  'Indiana' UNION SELECT T1.professional_id ,  T1.last_name ,  T1.cell_number FROM Professionals AS T1 JOIN Treatments AS

In [2]:
!git clone https://github.com/taoyds/test-suite-sql-eval.git

Cloning into 'test-suite-sql-eval'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 61 (delta 14), reused 11 (delta 10), pack-reused 39
Unpacking objects: 100% (61/61), 619.43 KiB | 1.26 MiB/s, done.


In [1]:
!pip3 install sqlparse
!pip3 install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
!python test-suite-sql-eval/evaluation.py --gold gold.txt --pred pred.txt --db /content/drive/MyDrive/spider/database --table /content/drive/MyDrive/spider/tables.json --etype all

easy pred: Gib die Ländercodes für Länder, in denen die Menschen Sprachen sprechen, die nicht
easy gold: SELECT DISTINCT CountryCode FROM countrylanguage WHERE LANGUAGE != "English"

medium pred: <unk>schema> Table: singer Columns: *, singer id,
medium gold: SELECT Birth_Year ,  Citizenship FROM singer

medium pred: <unk>schema> Table: city Columns: *, id, name
medium gold: SELECT Name ,  population ,  HeadOfState FROM country ORDER BY SurfaceArea DESC LIMIT 1

hard pred: Liste der Namen aller Gewinner, die sowohl 2013 als auch 2016 gespielt haben.
hard gold: SELECT winner_name FROM matches WHERE YEAR  =  2013 INTERSECT SELECT winner_name FROM matches WHERE YEAR  =  2016

hard pred: <unk>schema> Table: course Columns: *, course id,
hard gold: SELECT T3.Name ,  T2.Course FROM course_arrange AS T1 JOIN course AS T2 ON T1.Course_ID  =  T2.Course_ID JOIN teacher AS T3 ON T1.Teacher_ID  =  T3.Teacher_ID ORDER BY T3.Name

easy pred: SQL Server 2008 R2 - SQL Server 2008 R2 - SQL Server 2008 R

In [17]:
!python test-suite-sql-eval/evaluation.py --gold gold.txt --pred pred.txt --db /content/drive/MyDrive/spider/database --table /content/drive/MyDrive/spider/tables.json --etype all

easy pred: SELECT DISTINCT YEAR FROM automobiles AS t1 JOIN country
easy gold: select distinct year from cars_data where weight between 3000 and 4000;

medium pred: SELECT T1.name FROM Poker_players AS T1 JOIN players AS
medium gold: SELECT T1.Name FROM people AS T1 JOIN poker_player AS T2 ON T1.People_ID  =  T2.People_ID ORDER BY T2.Final_Table_Made

extra pred: SELECT T1.id, T2.last_name, T2.last
extra gold: SELECT professional_id ,  last_name ,  cell_number FROM Professionals WHERE state  =  'Indiana' UNION SELECT T1.professional_id ,  T1.last_name ,  T1.cell_number FROM Professionals AS T1 JOIN Treatments AS T2 ON T1.professional_id  =  T2.professional_id GROUP BY T1.professional_id HAVING count(*)  >  2

medium pred: SELECT T1.player_id, T1.player_id,
medium gold: SELECT first_name ,  last_name FROM players ORDER BY birth_date

medium pred: SELECT T1.Airline_ID FROM Flights AS T1 JOIN Flight
medium gold: SELECT count(*) FROM FLIGHTS AS T1 JOIN AIRLINES AS T2 ON T1.Airline  =  T2.u